# 強化学習と逐次意思決定問題の基礎

参考：
* [Foundations of Reinforcement Learning and Interactive Decision Making](https://arxiv.org/abs/2312.16730)

この資料では，逐次意思決定問題をMinimax問題として見ていきます．

* $\mathcal{M}$をモデルの集合
* $\mathrm{Alg}$をアルゴリズム
* $\operatorname{Perf}_T(\operatorname{Alg}, M)$を$T$ラウンド後の$M\in \mathcal{M}$における$\mathrm{Alg}$の性能

とします．
今回は，どんな$M \in \mathcal{M}$が来てもよく動くアルゴリズムを開発することを目指します．つまり，$\mathrm{Alg}$は次のMinimax問題の上界を与えることを目指します．

$$
\min _{\text {Alg }} \max _{M \in \mathcal{M}} \operatorname{Perf}_T(\mathrm{Alg}, M)
$$

## 統計的学習理論の復習

ここでは簡単に統計的学習問題について復習します．
次の問題を考えましょう：

* データが未知の分布$M^\star$からi.i.d.にサンプルされているとします：$\left(x^1, y^1\right), \ldots,\left(x^T, y^T\right) \in \mathcal{X} \times \mathcal{Y}$
* 良い予測器を作るのが目標です：$\widehat{f}: \mathcal{X} \rightarrow \mathcal{Y}^{\prime}$
* 予測器は何らかの損失関数$\ell: \mathcal{Y}^{\prime} \times \mathcal{Y} \rightarrow \mathbb{R}$によって評価されます．
* $\mathcal{H}^T:=\left\{\left(x^t, y^t\right)\right\}_{t=1}^T$をデータセットとします．
    * 統計的学習問題における「アルゴリズム」とは，データセットから予測器（関数）を返す写像のことです．つまり，$\widehat{f}\left(\cdot ; \mathcal{H}^T\right): \mathcal{X} \rightarrow \mathcal{Y}^{\prime}$のことです．

期待誤差$L(f):=\mathbb{E}_{(x, y) \sim M^{\star}}[\ell(f(x), y)]$とします．統計的学習問題における目標は，
$$
\mathbb{E}[L(\widehat{f})]
$$
が最小になるようなアルゴリズムを作ることです．ここで，$\mathbb{E}[\cdot]$はデータセット$\mathcal{H}^T$のサンプルについての期待値です．
しかし，ここで$\mathcal{F}$に何の仮定も置かない場合は，$T$が$|\mathcal{X}|$にスケールしない限り，良い予測器を作ることは期待できません（No-free-lunch定理と呼ばれることがあります．）

そこで，統計的学習理論では，何らかの予測器のクラス
$$\mathcal{F} \subseteq\{f: \mathcal{X} \rightarrow \mathcal{Y}\}$$
から，良い予測器を探すことを考えます．


### 経験リスク最小化と余剰誤差

次の表記を導入します：
* データセットに対する経験リスク $\widehat{L}(f)=\frac{1}{T} \sum_{i=1}^T \ell\left(f\left(x^i\right), y^i\right)$
* 経験リスクを最小化する予測器$\widehat{f} \in \underset{f \in \mathcal{F}}{\arg \min } \widehat{L}(f)$
* 余剰誤差（リグレット） $\mathcal{E}(f)=L(f)-\min _{f^{\prime} \in \mathcal{F}} L\left(f^{\prime}\right)$
    * 余剰誤差が小さい予測器$f$は，将来の出力を良く予測できることを意味しています．
* もし最適な予測器が$f^*(a) := \mathbb{E}[y \mid x=a] \in \mathcal{F}$を満たすとき，その統計的学習問題をrealizableである，と呼びます．

**余剰誤差と推定問題の関係** 

この余剰誤差の定義は，予測器のクラス$\mathcal{F}$とデータの分布$M^\star$の間に何の仮定も置いていない点で便利です（このような設定をAgnosticである，と呼ぶことがあります）．
$\mathcal{F}$がデータの分布を良く表すことができる場合（つまりrealizableの場合），また，損失関数として二乗誤差を考えると，余剰誤差は次のように書き換えることができます（推定問題と関連付けることができます）．

$$
\begin{aligned}
\mathcal{E}(f)&=
L(f)-\min _{f^{\prime} \in \mathcal{F}} L\left(f^{\prime}\right)\\
&=\mathbb{E}(f(x)-y)^2-\mathbb{E}\left(f^{\star}(x)-y\right)^2\\
&=\mathbb{E}_x\left(f(x)-f^{\star}(x)\right)^2+\underbrace{2 \mathbb{E}\left[\left(f^{\star}(x)-y\right)\left(f(x)-f^{\star}(x)\right)\right]}_{タワールールとか使えば０になると思う}\\
&= \mathbb{E}_x\left[\left(f(x)-f^{\star}(x)\right)^2\right]
\end{aligned}
$$

よって，realizableな場合は，「$f^*$と予測が近い予測器を得る」≒「汎化誤差が小さい予測器を得る」ことになります．

**経験リスク最小化について**

さて，汎化誤差を小さくする予測器をみつけるやり方として，経験リスク最小化があります．
経験リスク最小化は，特に$\mathcal{F}$が有限である場合は，良い結果が得られることが知られています．
次が成立します：

---

任意の有限な$\mathcal{F}$について，次が成立します：

$$
\mathbb{E}[\mathcal{E}(\widehat{f})] \lesssim \operatorname{comp}(\mathcal{F}, T)
$$

1. ここで，任意の有限な損失関数に対して，$\operatorname{comp}(\mathcal{F}, T)=\sqrt{\frac{\log |\mathcal{F}|}{T}}$
2. 二乗誤差関数かつrealizableであれば，$\operatorname{comp}(\mathcal{F}, T)=\frac{\log |\mathcal{F}|}{T}$

が成り立ちます．[MATH_statistical_learning_theory.ipynb](MATH_statistical_learning_theory.ipynb)に証明を書きました．

ちなみに，二乗誤差とconditional density estimationのどちらにおいても，$\frac{\log |\mathcal{F}|}{T}$を達成するアルゴリズムは存在します．

---

ここで，二乗根がついている収束レートをslow rate，２つ目のレートをfast rateと呼びます．
また，このレートは$|\mathcal{X}|$に依存していないことに注意しましょう．
つまり，何らかの制限された予測器のクラス$\mathcal{F}$を考えると，$\mathcal{X}$についての汎化性能を期待できます．
その意味で，$\log |\mathcal{F}|$は**capacity**や**expressiveness**と呼ばれたりします．

ちなみに無限集合に対応したい場合は，VC次元，Rademacher複雑度，Covering numberなどのテクニックがよく知られています．


## オンライン学習と予測

これまでは，何らかの分布からi.i.d.で出てきたデータセットを考えていました．
今度はオンライン学習（逐次予測）について考えてみましょう．具体的には，
* $T$個のサンプルを一気に受け取るのではなく，$(x^t, y^t)$を一つづつ受取り，各$x^t$では$y^t$ を予測することを目指します．
* また，サンプルは固定された分布に従うことを仮定しません．今回は敵対的になり得る状況を考えます．

オンライン学習では次の手続きを繰り返します：
1. For $t=1, \dots, T$で，
   1. これまでのデータ$\mathcal{H}^{t-1}=\left\{\left(x^1, y^1\right), \ldots,\left(x^{t-1}, y^{t-1}\right)\right\}$を使って，予測器$\widehat{f}^t=\widehat{f}^t\left(\cdot \mid \mathcal{H}^{t-1}\right)$を計算
   2. $\left(x^t, y^t\right) \in \mathcal{X} \times \mathcal{Y}$を観測

オンライン学習の目標は，次の損失の和の最小化です：

$$
\sum_{t=1}^T \ell\left(\widehat{f}^t\left(x^t\right), y^t\right)
$$

各ステップでは，今までのデータを使って予測器を作っているので，この損失の和は「out-of-sampleについての予測誤差の和」と考えることができます．
ところで今回はデータの分布に何の仮定も置いていないので，損失の和について何か言うことは難しいです．そこで，次のリグレットを考えましょう：
$$
\mathbf{R e g}=\sum_{t=1}^T \ell\left(\widehat{f}^t\left(x^t\right), y^t\right)-\min _{f \in \mathcal{F}} \sum_{t=1}^T \ell\left(f\left(x^t\right), y^t\right)
$$

この章の目的は，**任意の**データ系列について，リグレットを小さくするような予測器を出すアルゴリズムを作ることです．一見すると，任意の系列に対してリグレットを小さくすることはできないきがしますが，大丈夫，できます．
このような任意系列に対するオンライン学習アルゴリズムは，後で学ぶ意思決定問題で利用するときに便利です．意思決定問題では「過去の決定」に応じてデータがでてきますが，これによってi.i.d.でデータが生成されることは期待できません．そういう場合でオンライン学習のアルゴリズムが役に立ちます．

---

**「プロパー」な学習，確率的な予測**

オンライン学習では，$\widehat{f}^t \in \mathcal{F}$であることを要求しません．
$\widehat{f}^t \in \mathcal{F}$の場合な手法を「プロパー」と呼びます（そうでない場合は「プロパーでない」と呼びます）．

また，次のような**確率的な予測器**（randomized prediction）を使うとしばしば便利です．
$$
\widehat{f^t} \sim q^t\left(\cdot \mid \mathcal{H}^{t-1}\right)
$$

ここで，$q^t$は関数についての分布です（たいていの場合は$\mathcal{F}$上の分布です）．
このような確率的な予測器を考える場合は，次のリグレットを使います：

$$
\mathbf{R e g}=\sum_{i=1}^T \mathbb{E}_{\widehat{f}^t \sim q^t}\left[\ell\left(\widehat{f}^t\left(x^t\right), y^t\right)\right]-\min _{f \in \mathcal{F}} \sum_{i=1}^T \ell\left(f\left(x^t\right), y^t\right)
$$

以降では，サンプルが敵対的に出てくるような問題について考えます．
すなわち，
* 決定的なアルゴリズムに対しては，$(x^t, y^t)$が過去のデータと$\hat{f}^t$をもとに選択される問題
* 確率的なアルゴリズムに対しては，$(x^t, y^t)$が過去のデータと$\hat{q}^t$をもとに選択される問題

について考えていきます．

注意：これはまだ意思決定問題ではありません．なぜなら，生成した予測器によって，データの生成のされ方が**直接的に**影響されるわけではないので．

---

**オンライン学習と統計的学習のつながり**

オンライン学習は統計的学習を一般化しています．実際，オンライン学習に使われるアルゴリズムは，「online-to-batch conversion」と呼ばれるテクニックによって，統計的学習用のアルゴリズムに変形することができます．
これは次の補題で定式化されます：

---

$\left(x^1, y^1\right), \ldots,\left(x^T, y^T\right)$を，分布$M^*$からi.i.d.にサンプルされたデータとします．
損失関数$a \mapsto \ell(a, b)$を，任意の$b$に対して，最初の引数について凸な関数とします．
このとき，任意のオンライン学習アルゴリズムに対して，
$$
\widehat{f}(x)=\frac{1}{T} \sum_{t=1}^T \widehat{f}^t(x)
$$
とすると，
$$
\mathbb{E}[\mathcal{E}(\widehat{f})] \leq \frac{1}{T} \cdot \mathbb{E}[\mathbf{R e g}]
$$
が成り立ちます．

**証明**

$(x, y) \sim M^{\star}$を，$\mathcal{H}^T$に対して独立なサンプルとします．すると，Jensenの不等式から，

$$
\mathbb{E}[L(\widehat{f})]=\mathbb{E}\left[\mathbb{E}_{(x, y)} \ell\left(\frac{1}{T} \sum_{t=1}^T \widehat{f}^t(x), y\right)\right] \leq \mathbb{E}\left[\frac{1}{T} \sum_{t=1}^T \mathbb{E}_{(x, y)} \ell\left(\widehat{f}^t(x), y\right)\right]
$$

が成立します．また，$\hat{f}^t$が$\mathcal{H}^{t-1}$の関数であり，$(x, a)$と$(x^t, y^t)$がi.i.d.なので，

$$
\mathbb{E}[L(\widehat{f})]=\mathbb{E}\left[\mathbb{E}_{(x, y)} \ell\left(\frac{1}{T} \sum_{t=1}^T \widehat{f}^t(x), y\right)\right] \leq \mathbb{E}\left[\frac{1}{T} \sum_{t=1}^T \mathbb{E}_{(x, y)} \ell\left(\widehat{f}^t(x), y\right)\right]
=\mathbb{E}\left[\frac{1}{T} \sum_{t=1}^T \mathbb{E}_{\left(x^t, y^t\right)} \ell\left(\widehat{f}^t\left(x^t\right), y^t\right)\right]
$$
も成り立ちます．
また，
$$
\min _{f \in \mathcal{F}} L(f)=\min _{f \in \mathcal{F}} \mathbb{E}\left[\frac{1}{T} \sum_{t=1}^T \ell\left(f\left(x^t\right), y^t\right)\right] \geq \mathbb{E}\left[\min _{f \in \mathcal{F}} \frac{1}{T} \sum_{t=1}^T \ell\left(f\left(x^t\right), y^t\right)\right]
$$
も成り立つので，

$$
\mathbb{E}[\mathcal{E}(\widehat{f})] = 
\mathbb{E}[L(\widehat{f})] - \min _{f \in \mathcal{F}} L(f)
\leq 
\mathbb{E}\left[\frac{1}{T} \sum_{t=1}^T \mathbb{E}_{\left(x^t, y^t\right)} \ell\left(\widehat{f}^t\left(x^t\right), y^t\right)\right]
- \mathbb{E}\left[\min _{f \in \mathcal{F}} \frac{1}{T} \sum_{t=1}^T \ell\left(f\left(x^t\right), y^t\right)\right]
=\frac{1}{T} \cdot \mathbb{E}[\mathbf{R e g}]
$$

が成立します．

---

上の補題から，リグレットはi.i.d.データに対しての余剰誤差の一般化であると見ることができます．
また，最初に見たように，realizableな問題については，リグレットを推定問題と関連付けることができます．


---

$x^1, \ldots, x^T$を，任意の方法で生成されたデータとします．
また，各$y^t$は，次の期待値を持つようにランダムに生成されたものとします：
$$\mathbb{E}\left[y^t \mid x^t=x\right]=f^{\star}(x)$$

このとき，二乗誤差を考えると，
$$
\mathbb{E}[\mathbf{R e g}] \geq \mathbb{E}\left[\sum_{t=1}^T\left(\widehat{f^t}\left(x^t\right)-f^{\star}\left(x^t\right)\right)^2\right]
$$

が成立します．

---

この結果は，$x^1, \ldots, x^T$が敵対的に生成されても成立することに注意しましょう．統計的学習問題ではi.i.d.に生成されていたので，こちらのほうが一般的です．



### Exponential Weights Algorithm

オンライン学習で基本のアルゴリズムが，Exponential Weights Algorithmです．これは有限な$\mathcal{F}$に使うことができます．
各$t$で，アルゴリズムは

$$
q^t(f) \propto \exp \left\{-\eta \sum_{i=1}^{t-1} \ell\left(f\left(x^i\right), y^i\right)\right\}
$$

であるような分布$q^t \in \Delta(\mathcal{F})$を計算します．ここで，$\eta$は学習率です．
これを基本として，次のAverage版のアルゴリズムと，Randomizedなアルゴリズム の両方を導入します．

![ExpWeight](figs/ExponentialWeighting.png)

２つのアルゴリズムの違いは，予測器の出し方の違いだけですね．つまり，

$$
\widehat{f}^t=\mathbb{E}_{f \sim q^t}[f], \quad \text { or } \quad \widehat{f}^t \sim q^t
$$

の違いだけです．
ここで，Average版のアルゴリズムは損失関数の「中」で期待値を取り，一方でRandomized版では損失関数の「外」で期待値を取っています．
Average版はそのため，損失関数の構造（凸性など）を利用して高速な収束を実現することがあります．
Exponential Weights Algorithmの収束レートを証明してみましょう．（これは統計的学習のレートに似てることに注意しましょう．）

---

任意の有限な$\mathcal{F}$について，Exponential Weights Algorithmは，適切な$\eta$のとき，
$$
\frac{1}{T} \mathbf{R e g} \lesssim \operatorname{comp}(\mathcal{F}, T)
$$
が達成されます．ここで，
1. 任意の有限な損失関数については，Randomized版のアルゴリズムが，$\operatorname{comp}(\mathcal{F}, T)=\sqrt{\frac{\log |\mathcal{F}|}{T}}$を達成します．
2. 「二乗損失の回帰問題」もしくは「対数損失のconditional density estimation」については，Average版のアルゴリズムが，$\operatorname{comp}(\mathcal{F}, T)=\frac{\log |\mathcal{F}|}{T}$を達成します．

**証明**

これを証明するには，統計的学習でやったようなやり方は使えません．そのため，ポテンシャル関数を使った償却解析を考えることが一般的です．
一般に，次の３ステップによって証明します：

1. ポテンシャル関数を定義
2. ポテンシャル関数の増え方と損失関数の関係を導出
3. 任意の$f \in \mathcal{F}$の累積損失を，最終的なポテンシャルに関連させる

* [償却解析](https://en.wikipedia.org/wiki/Amortized_analysis)の補足（[これ](https://catupper.hatenablog.com/entry/20131207/1386402713)も参考になるかも．）：アルゴリズム解析の文脈で，アルゴリズムの償却計算量を求めることを償却解析といいます．償却計算量とは，連続して処理を実行するアルゴリズムの，その平均的な処理の計算量のことです．今回のオンライン学習では，連続して処理を実行するアルゴリズムのリグレットを解析しているので，リグレットの償却解析です．
償却解析には色々やり方がありますが，ポテンシャル法がよく使用されます．ポテンシャル法では，処理の対象であるデータの構造に対して値を対応付ける**ポテンシャル関数**を使用します．途中のポテンシャル関数が互いに打ち消し合うように設計することで，累積和ではなく，アルゴリズムの実行前と実行後のポテンシャル関数の結果だけ見れば解析ができます．

Exponential Weights Algorithmでは，次のポテンシャル関数を使います：

$$
\Phi_\eta^t=-\log \sum_{f \in \mathcal{F}} \exp \left\{-\eta \sum_{i=1}^t \ell\left(f\left(x^i\right), y^i\right)\right\}
$$


**対数損失かつconditional density estimationのとき**

この場合のリグレットを証明しましょう．
このとき，任意の$x$について，$f(x)$は$\mathcal{Y}$上の分布であり，$\ell_{\log }(f(x), y)=-\log f(y \mid x)$です．

$\eta=1$のとき，Exponential Weights Algorithmでは次の重みを計算します：

$$
\widehat{f}^t\left(y^t \mid x^t\right)=\sum_{f \in \mathcal{F}} q^t(f) f\left(y^t \mid x^t\right)=\sum_{f \in \mathcal{F}} f\left(y^t \mid x^t\right) \frac{\exp \left\{-\sum_{i=1}^{t-1} \ell_{\log }\left(f\left(x^i\right), y^i\right)\right\}}{\sum_{f \in \mathcal{F}} \exp \left\{-\sum_{i=1}^{t-1} \ell_{\log }\left(f\left(x^i\right), y^i\right)\right\}}
$$

よって，

$$
\ell_{\log }\left(\widehat{f}\left(x^t\right), y^t\right)=-\log \widehat{f}^t\left(y^t \mid x^t\right)=\Phi_1^t-\Phi_1^{t-1}
$$

が成り立ちます．Telescopingより，
$$
\sum_{t=1}^T \ell_{\log }\left(\widehat{f}\left(x^t\right), y^t\right)=\Phi_1^T-\Phi_1^0
$$
が成立します．また，$\Phi_1^0=-\log|\mathcal{F}|$であることに気をつけましょう．
さらに，log-sum-expの不等式を使うと，
$$
\Phi_1^T=-\log \sum_{f \in \mathcal{F}} \exp \left\{-\sum_{i=1}^T \ell\left(f\left(x^i\right), y^i\right)\right\}
\leq
\sum_{i=1}^T \ell\left(f^*\left(x^i\right), y^i\right)
$$
が任意の$f^*$で成立します．以上の結果を合わせれば，


$$
\begin{aligned}
\mathbf{R e g}
&=
\sum_{t=1}^T \ell_{\log }\left(\widehat{f}\left(x^t\right), y^t\right)
-
\sum_{t=1}^T \ell_{\log }\left(f^*\left(x^t\right), y^t\right)\\
&=\Phi_1^T+\log|\mathcal{F}|
-
\sum_{t=1}^T \ell_{\log }\left(f^*\left(x^t\right), y^t\right)
\leq \log|\mathcal{F}|
\end{aligned}
$$

が成り立ちます．あとは$T$で割れば証明終わりです．

---

**有限な損失関数の場合**

続いて，有限な損失関数についての証明をしましょう．
一般性を失わずに，$\ell \circ f \in[0,1]$ for all $f \in \mathcal{F}$のケースを考えます．
さて，Randomized版では次が成立します．

$$
\mathbb{E}_{\widehat{f}^t \sim q^t}\left[\ell\left(\widehat{f}^t\left(x^t\right), y^t\right)\right]=\sum_{f \in \mathcal{F}} \ell\left(f\left(x^t\right), y^t\right) \frac{\exp \left\{-\eta \sum_{i=1}^{t-1} \ell\left(f\left(x^i\right), y^i\right)\right\}}{\sum_{f \in \mathcal{F}} \exp \left\{-\eta \sum_{i=1}^{t-1} \ell\left(f\left(x^i\right), y^i\right)\right\}}
$$

さて，次の[Hoeffdingの補題](https://en.wikipedia.org/wiki/Hoeffding%27s_lemma)を使います：
確率変数$a \leq X\leq b$を考えます．
任意の$\eta \in \mathbb{R}$について，

$$
\mathbb{E}\left[e^{\eta X}\right] \leq \exp \left(\eta \mathbb{E}[X]+\frac{\eta ^2(b-a)^2}{8}\right)
$$

Hoefdingの不等式を使えば，

$$
\eta \mathbb{E}_{\widehat{f}^t \sim q^t}\left[\ell\left(\widehat{f}^t\left(x^t\right), y^t\right)\right] \leq-\log \sum_{f \in \mathcal{F}} \frac{\exp \left\{-\eta \sum_{i=1}^{t-1} \ell\left(f\left(x^i\right), y^i\right)\right\}}{\sum_{f \in \mathcal{F}} \exp \left\{-\eta \sum_{i=1}^{t-1} \ell\left(f\left(x^i\right), y^i\right)\right\}}\exp \left\{-\eta \ell\left(f\left(x^t\right), y^t\right)\right\} +\frac{\eta^2}{8}
$$

この右辺は
$$
\Phi_\eta^t-\Phi_\eta^{t-1}+\frac{\eta^2}{8}
$$
と同じであることに注意しましょう．よって，$T$で和を取れば，

$$
\eta \sum_{t=1}^T \mathbb{E}_{\widehat{f}^t \sim q^t}\left[\ell\left(\widehat{f}^t\left(x^t\right), y^t\right)\right] \leq \Phi_\eta^T-\Phi_\eta^0+\frac{T \eta^2}{8}
$$

が成立します．
そして，対数損失の場合でやったように，log-sum-expの不等式を使うと，任意の$f^*$について
$$\Phi_\eta^T \leq \eta \sum_{t=1}^T \ell\left(f^{\star}\left(x^t\right), y^t\right)$$
が成立します．よって，

$$
\sum_{t=1}^T \mathbb{E}_{\widehat{f}^t \sim q^t}\left[\ell\left(\widehat{f}^t\left(x^t\right), y^t\right)\right]-\ell\left(f^{\star}\left(x^t\right), y^t\right) \leq \frac{T \eta}{8}+\frac{\log |\mathcal{F}|}{\eta}
$$

であり，$\eta=\sqrt{\frac{8 \log \mid \mathcal{F}}{T}}$とすれば，

$$
\sum_{t=1}^T \mathbb{E}_{\widehat{f^t} \sim q^t}\left[\ell\left(\widehat{f}^t\left(x^t\right), y^t\right)\right]-\ell\left(f^{\star}\left(x^t\right), y^t\right) \leq \sqrt{\frac{T \log |\mathcal{F}|}{2}}
$$

が成立します．

---

**補足**

ちなみに，二乗誤差についての補題を証明していませんが，二乗誤差については"mixable loss"を使うことで，対数損失から一発で言及することができます．
詳細は教科書のRemark 6を参照してください．

また，このExponential Weights AlgorithmはMirror descentによって，Online linear optimizationを解いている，とみなすこともできます．
詳細は教科書のRemark 7を参照してください．


## 確率的バンディット問題

基本的な統計的学習やオンライン学習の理論をおさらいしたところで，バンディット問題に進みます．
バンディット問題では次の処理を繰り返します．
1. 意思決定：$\pi^t \in \Pi:=\{1, \ldots, A\}$.
2. 報酬の観測：$r^t \in \mathbb{R}$.

ここで，$\Pi$はdecision spaceもしくは行動空間と呼びます．
また，意思決定はrandomizeすることもできます．つまり，
$p^t=p^t\left(\cdot \mid \mathcal{H}^{t-1}\right)$として， $pi^t \sim p^t$なる意思決定を考えても良いです．

さて，まずは確率的バンディットについて考えましょう．確率的バンディットでは，次を仮定します．

$$
r^t \sim M^{\star}\left(\cdot \mid \pi^t\right)
$$

また，$f^{\star}(\pi):=\mathbb{E}[r \mid \pi]$と表記します．これは$r \sim M^{\star}(\cdot \mid \pi)$の期待値です．

確率的バンディットについて，次のようなリグレットを考えていきます：

$$\boldsymbol{R e g}:=\sum_{t=1}^T f^{\star}\left(\pi^{\star}\right)-\sum_{t=1}^T \mathbb{E}_{\pi^t \sim p^t}\left[f^{\star}\left(\pi^t\right)\right]$$

確率的バンディットでの基本的な目標は次のように，劣線形なリグレットを達成するアルゴリズムを設計することになります：

$$
\frac{\mathbb{E}[\text { Reg }]}{T} \rightarrow 0 \quad \text { as } \quad T \rightarrow \infty
$$

バンディット問題では，オンライン学習問題と異なり，選択したアームについての情報しか与えません．つまり，$\pi \neq \pi^t$なるアームの情報は何も得られません．
このような難しさは，**partial feedback**もしくは**bandit feedback**と呼ばれます．
敵対的なオンライン学習問題とは難しさの種類が違うことに注意しましょう．リグレットを比較するとわかりやすいかも？

**補足：その他のリグレット**

次のような経験リグレットを考えることがあります．

$$
\max _{\pi \in \Pi} \sum_{t=1}^T r^t(\pi)-\sum_{t=1}^T r^t\left(\pi^t\right)
$$

ここで，$\pi \neq \pi^t$について，$r^t(\pi)$は「反事実的な報酬」と呼ばれることがあります．また，Hoeffdingの不等式を使えば，この経験リグレットは最初の期待リグレットについて$\mathcal{O}(\sqrt{T})$のオーダーしか変わらないことが示せます．




### 探索の必要性

統計的学習用では，データに最もフィットする関数を選択し続ければ，余剰誤差が小さくなることを学びました．
一方で，バンディットではそうはいきません．
バンディットでも同じようなことをすると（つまり，データに最もフィットする関数を選択し続ける），線形なリグレットになるようなバンディットを作れてしまいます．

探索を実現するアルゴリズムとして，UCBアルゴリズムを紹介します．

**UCBアルゴリズム**

UCBアルゴリズムでは，「Optimism in the face of uncertainty」の原則を利用します．
このOFUの原則を実現するために，各時刻$t$で，次の**信頼区間**を構築することを考えます．

$$
\underline{f}^t, \bar{f}^t: \Pi \rightarrow \mathbb{R}
$$
ただし，この２つは次を満たすとします：
$$
\forall t \in[T], \pi \in \Pi, \quad f^*(\pi) \in\left[\underline{f}^t(\pi), \bar{f}^t(\pi)\right]
$$

$\underline{f}^t$はlower confidence bound，$\bar{f}^t$はupper confidence bound と呼びます．（次の図をイメージするとわかりやすいです）

![confidence](figs/confidence-bound.png)

UCBアルゴリズムは，この信頼区間において，最も楽観的な行動を選択するようにします．
すなわち，

$$
\pi^t=\underset{\pi \in \Pi}{\arg \max } \bar{f}^t(\pi)
$$

です．このとき，次の補題が示すように，即時リグレットを信頼区間によって抑えることができます．

---

**補題**

固定された$t$において，$f^{\star}(\pi) \in\left[\underline{f}^t(\pi), \bar{f}^t(\pi)\right]$ for all $\pi$ を仮定します．
このとき，
$\pi^t=\underset{\pi \in \Pi}{\arg \max } \bar{f}^t(\pi)$によって行動を選択すると，
$$
f^{\star}\left(\pi^{\star}\right)-f^{\star}\left(\pi^t\right) \leq \bar{f}^t\left(\pi^t\right)-f^{\star}\left(\pi^t\right) \leq \bar{f}^t\left(\pi^t\right)-\underline{f}^t\left(\pi^t\right) .
$$

が成立します．（証明は簡単なので省略）

---

よって，リグレットを抑えるためには，信頼区間が十分な速度で小さくなれば良いわけですね．
Hoeffdingの不等式を使えば，$r^t \in[0,1]$である限り，確率$1-\delta$以上で，任意の$t\in [T]$に対して，
$$
\left|\widehat{f}^t(\pi)-f^{\star}(\pi)\right| \leq \sqrt{\frac{2 \log \left(2 T^2 A / \delta\right)}{n^t(\pi)}}
$$
が成立します．ここで，$\hat{f}^t$は経験推定値であり，$n^t(\pi):=\sum_{i<t} \mathbb{I}\left\{\pi^i=\pi\right\}$です．
これより，次の信頼区間を考えれば，信頼区間の条件は満たされます：

$$
\bar{f}^t(\pi)=\widehat{f}^t(\pi)+\sqrt{\frac{2 \log \left(2 T^2 A / \delta\right)}{n^t(\pi)}}, \quad \text { and } \quad \underline{f}^t(\pi)=\widehat{f}^t(\pi)-\sqrt{\frac{2 \log \left(2 T^2 A / \delta\right)}{n^t(\pi)}} \text {, }
$$

以上を踏まえて，UCBアルゴリズムのリグレットバウンドを導出してみましょう．

---

**UCBのリグレットバウンド**

上で言及した信頼区間を使った上で，$f^{\star}(\pi) \in\left[f^t(\pi), \bar{f}^t(\pi)\right]$ が任意の$t \in[T]$ と$\pi \in \Pi$で成立している状況を考えます．

このとき，上で見たように，次のリグレットバウンドが成立します：

$$
\sum_{t=1}^T f^{\star}\left(\pi^{\star}\right)-f^{\star}\left(\pi^t\right) \leq \sum_{t=1}^T \bar{f}^t\left(\pi^t\right)-\underline{f}^t\left(\pi^t\right)=\sum_{t=1}^T 2 \sqrt{\frac{2 \log \left(2 T^2 A / \delta\right)}{n^t\left(\pi^t\right)}} \wedge 1
$$

この右辺をバウンドするために，Potential argumentってやつを使います．

---

**Confidence width potential lemma**

次が成立します．

$$
\sum_{t=1}^T \frac{1}{\sqrt{n^t\left(\pi^t\right)}} \wedge 1 \lesssim \sqrt{A T}
$$

* TODO: 信頼区間の幅をポテンシャル関数とみなせば，償却解析における「ポテンシャル関数の和のバウンド」みたいな形をしてるので，これもPotential lemmaって呼んでいるのかも？Online解析の話を調べればでてくるかも．

**証明**

まず，
$$
\sum_{t=1}^T \frac{1}{\sqrt{n^t\left(\pi^t\right)}} \wedge 1=\sum_\pi \sum_{t=1}^T \frac{\mathbb{I}\left\{\pi^t=\pi\right\}}{\sqrt{n^t(\pi)}} \wedge 1=\sum_\pi \sum_{t=1}^{n^{T+1}(\pi)} \frac{1}{\sqrt{t-1}} \wedge 1
$$

が成立します．簡単な計算で，$\sum_{t=1}^n \frac{1}{\sqrt{t-1}} \wedge 1 \leq 1+2 \sqrt{n}$であることがわかります．これを使うと，
$$
\sum_{t=1}^T \frac{1}{\sqrt{n^t\left(\pi^t\right)}} \wedge 1
=\sum_\pi \sum_{t=1}^{n^{T+1}(\pi)} \frac{1}{\sqrt{t-1}} \wedge 1
\leq A+2 \sum_\pi \sqrt{n^T(\pi)}
$$
が成立します．また，Jensenの不等式から
$$
\sum_\pi \sqrt{n^T(\pi)} \leq A \sqrt{\sum_\pi \frac{n^T(\pi)}{A}}=A \sqrt{T / A}=\sqrt{A T}
$$
です．あとは合体したら証明完了です．

---

まとめると，基本的な証明の戦略は：
1. 信頼区間とOptimismを利用して，リグレットを信頼区間の幅でバウンドする
2. potential argumentを使って信頼区間の幅の累積和が小さくなることを示す
 
このやり方は文脈付きバンディットでも強化学習でもほぼ同じです．

## ベイジアンバンディット

ここまで見たのは頻度主義的なアプローチでした．今度はベイズ主義的なアプローチを見てみましょう．
つまり，背後にあるモデルが，何らかの事前分布$\mu \in \Delta(\mathcal{M})$から出てくることを仮定します（ここで，$\mu$は既知です．そうでなければ，頻度主義的なアプローチと変わりません）．

このような問題設定では，次のベイズリグレットを考えます：

$$
\boldsymbol{\operatorname { R e g }}_{\text {Bayes }}(\mu):=\mathbb{E}_{M^{\star} \sim \mu} \mathbb{E}^{M^{\star}}[\mathbf{R e g}],
$$

このように，事前知識がある場合には，頻度主義的なアプローチとは違うアルゴリズムを使います．
有名な手法に，Posterior sampling（Thompson sampling もしくは probability matching とも呼ばれます）があります．

Posterior samplingでは次の処理を繰り返します．
1. Set $p^t(\pi)=\mathbb{P}\left(\pi^{\star}=\pi \mid \mathcal{H}^{t-1}\right)$, where $\mathcal{H}^{t-1}=\left(\pi^1, r^1\right), \ldots,\left(\pi^{t-1}, r^{t-1}\right)$. 
    * 今までのデータ$\mathcal{H}^{t-1}$を使って，最適な行動$\pi^\star$の事後分布を作ります．
2. Sample $\pi^t \sim p^t$ and observe $r^t$.
    * 行動を事後分布からサンプルすることで，学習者の行動を$\pi^\star$の事後分布に「マッチ」させます．

**Posterior sampling のリグレット解析**

今回は簡単のために，
* 真のモデルが$M^{\star}(\cdot \mid \pi)=\mathcal{N}\left(f^{\star}(\pi), 1\right)$の形式であるとします．よって，$M^\star$の代わりに$f^\star$を見つければ十分です．
* $f^\star$は既知のクラス$\mathcal{F}$に含まれているとします．
* モデルの事前分布として，$\mu \in \Delta(\mathcal{F})$を考えます．

このとき，次のリグレットバウンドを導出しましょう．（ちゃんとしたバウンドは後で導出します．ここでは証明の流れを抑えるために簡易版を出します）

$$
\boldsymbol{\operatorname { R e g }}_{\text {Bayes }}(\mu) \lesssim \sqrt{A T \log |\mathcal{F}|}
$$

次の表記を導出しましょう．
* $\mathbb{P}$：$f^\star \sim \mu$とバンディットアルゴリズムの実行によって誘導される，$(f^\star, \mathcal{H}^T)$上の確率測度（押し出し測度，law of ...とも呼ばれます）を$\mathbb{P}$とします．
* $\mathbb{E}[\cdot]$ は，その確率測度上の期待値とします．
* また，$\mathbb{E}_t[\cdot]=\mathbb{E}\left[\cdot \mid \mathcal{H}^t\right]$ and $\mathbb{P}_t[\cdot]=\mathbb{P}\left[\cdot \mid \mathcal{H}^t\right]$とします．
* $\mu^t(f):=\mathbb{P}\left(f^{\star}=f \mid \mathcal{H}^{t-1}\right)$を，$f^\star$の事後分布とします．

タワールールを使うと，リグレットは次で書き換えることができます．
$$
\boldsymbol{\operatorname { R e g }}_{\text {Bayes }}(\mu)=\mathbb{E}\left[\sum_{t=1}^T \mathbb{E}_{t-1}\left[f^{\star}\left(\pi_{f^{\star}}\right)-f^{\star}\left(\pi^t\right)\right]\right]
$$

ここで，$\pi^{\star}=\pi_{f^{\star}}$としました．これは確率変数$f^\star$に依存するので，$\pi^\star$も確率変数です．

学習者の意思決定$\pi^t$は，$\mathcal{H}^{t-1}$のもとで，$f^\star$とは条件付き独立であることに注意しましょう（アルゴリズムの仕組みを考えればわかります）．
これを使うと，リグレットの中身は次のように分解することができます：
$$
\begin{aligned}
\mathbb{E}_{t-1}\left[f^{\star}\left(\pi_{f^{\star}}\right)-f^{\star}\left(\pi^t\right)\right]
&=\mathbb{E}_{f^{\star} \sim \mu^t, \pi^t \sim p^t}\left[f^{\star}\left(\pi_{f^{\star}}\right)-f^{\star}\left(\pi^t\right)\right]\\
&= \mathbb{E}_{f^{\star} \sim \mu^t, \pi^t \sim p^t}\left[f^{\star}\left(\pi_{f^{\star}}\right)-\bar{f}^t\left(\pi^t\right)\right]\\
&= \mathbb{E}_{f^{\star} \sim \mu^t}\left[f^{\star}\left(\pi_{f^{\star}}\right)-\bar{f}^t\left(\pi_{f^{\star}}\right)\right]
\end{aligned}
$$

が成り立ちます．
ここで二行目は$\bar{f}^t(\pi)=\mathbb{E}_{f^{\star} \sim \mu^t}\left[f^{\star}(\pi)\right]$を使い，
三行目は$\pi^t\sim p^t$が$f^\star \sim \mu^t$のもとでは$\pi_{f^\star}$の従う分布と同じになることを使っています．

この最後の項は，「$\pi_{f^\star}$のもとで，$f^\star$がどれくらい$\bar{f}^t$から離れているか？」の平均的な量を表しています．
これは一見すると学習者の意思決定に関係なさそうな量ですが，次の補題を使うことで，意思決定と関係させることができます．

---

**補題（Decoupling）**

任意の関数$\bar{f}: \Pi \rightarrow \mathbb{R}$について，

$$
\mathbb{E}_{f^{\star} \sim \mu^t}\left[f^{\star}\left(\pi_{f^{\star}}\right)-\bar{f}\left(\pi_{f^{\star}}\right)\right] \leq \sqrt{A \cdot \mathbb{E}_{f^{\star} \sim \mu^t} \mathbb{E}_{\pi^t \sim p^t}\left[\left(f^{\star}\left(\pi^t\right)-\bar{f}\left(\pi^t\right)\right)^2\right]}
$$

**証明**

これはより一般的な結果を使って証明します．
任意の$\nu \in \Delta(\mathcal{F})$および
$\bar{f}: \Pi \rightarrow \mathbb{R}$に対して，$p(\pi)=\mathbb{P}_{f \sim \nu}\left(\pi_f=\pi\right)$とすると，

$$
\mathbb{E}_{f \sim \nu}\left[f\left(\pi_f\right)-\bar{f}\left(\pi_f\right)\right] \leq \sqrt{A \cdot \mathbb{E}_{f \sim \nu} \mathbb{E}_{\pi \sim p}\left[(f(\pi)-\bar{f}(\pi))^2\right]}
$$

が成立することを示しましょう．

* 補足：左辺では$f$と$\pi_f$がcoupledである一方で，右辺では$\pi$は$p$（$\pi_f$についての分布）からサンプルされるので，$f$と$\pi$は独立しています．その意味で，右辺はdecoupledです．

これを証明するために，コーシー・シュワルツの不等式を使います：

$$
\begin{aligned}
\mathbb{E}_{f \sim \nu}\left[f\left(\pi_f\right)-\bar{f}\left(\pi_f\right)\right] & =\mathbb{E}_{f \sim \nu}\left[\frac{p^{1 / 2}\left(\pi_f\right)}{p^{1 / 2}\left(\pi_f\right)}\left(f\left(\pi_f\right)-\bar{f}\left(\pi_f\right)\right)\right] \\
& \leq\left(\mathbb{E}_{f \sim \nu}\left[\frac{1}{p\left(\pi_f\right)}\right]\right)^{1 / 2} \cdot\left(\mathbb{E}_{f \sim \nu}\left[p\left(\pi_f\right)\left(f\left(\pi_f\right)-\bar{f}\left(\pi_f\right)\right)^2\right]\right)^{1 / 2}
\end{aligned}
$$

一項目が
$$
\mathbb{E}_{f \sim \nu}\left[\frac{1}{p\left(\pi_f\right)}\right]=\sum_f \frac{\nu(f)}{p\left(\pi_f\right)}=\sum_\pi \sum_{f: \pi_f=\pi} \frac{\nu(f)}{p(\pi)}=\sum_\pi \frac{p(\pi)}{p(\pi)}=A .
$$
であり，二項目が
$$
\mathbb{E}_{f \sim \nu}\left[p\left(\pi_f\right)\left(f\left(\pi_f\right)-\bar{f}\left(\pi_f\right)\right)^2\right] \leq \mathbb{E}_{f \sim \nu}\left[\sum_\pi p(\pi)(f(\pi)-\bar{f}(\pi))^2\right]=\mathbb{E}_{f \sim \nu} \mathbb{E}_{\pi \sim p}\left[(f(\pi)-\bar{f}(\pi))^2\right]
$$

であることを利用すれば，証明完了です．

---

このDecouplingの補題を使えば，ベイズリグレットを次のように抑えることができます．
$$
\begin{aligned}
\mathbb{E}[\mathbf{R e g}] & \leq \mathbb{E}\left[\sum_{t=1}^T \sqrt{A \cdot \mathbb{E}_{f^{\star} \sim \mu^t} \mathbb{E}_{\pi^t \sim p^t}\left[\left(f^{\star}\left(\pi^t\right)-\bar{f}^t\left(\pi^t\right)\right)^2\right]}\right] \\
& \leq \sqrt{A T \cdot \mathbb{E}\left[\sum_{t=1}^T \mathbb{E}_{f^{\star} \sim \mu^t} \mathbb{E}_{\pi^t \sim p^t}\left[\left(f^{\star}\left(\pi^t\right)-\bar{f}^t\left(\pi^t\right)\right)^2\right]\right]} .
\end{aligned}
$$

最後に，$\sum_{t=1}^T \mathbb{E}_{f^{\star} \sim \mu^t} \mathbb{E}_{\pi^t \sim p^t}\left[\left(f^{\star}\left(\pi^t\right)-\bar{f}^t\left(\pi^t\right)\right)^2\right] \leq \log |\mathcal{F}|$であることを示しましょう．

これをするためには，次の表記を導入します．

* エントロピー：$\mathbb{P}$上の確率変数$X$について，$\operatorname{Ent}(X) \equiv \operatorname{Ent}(\mathbb{P}):=\sum_x p(x) \log (1 / p(x))$
* 条件付きエントロピー：確率変数$X$と$Y$について，$\operatorname{Ent}(X \mid Y=y):=\operatorname{Ent}\left(\mathbb{P}_{X \mid Y=y}\right)$および，$\operatorname{Ent}(X \mid Y):=\mathbb{E}_{y \sim p_Y}[\operatorname{Ent}(X \mid Y=y)]$
* KLダイバージェンス：分布$\mathbb{P}$と$\mathbb{Q}$について，$D_{\mathrm{KL}}(\mathbb{P} \| \mathbb{Q})=\sum_x p(x) \log (p(x) / q(x))$

表記の簡単のため，以降では対象とする確率変数を太字で，実現されたサンプルは細字で書くことにします．
これから，$\operatorname{Ent}\left(\boldsymbol{f}^{\star} \mid \mathcal{H}^t\right)$が**ポテンシャル関数**であることを示していきます．

教科書のLemma 39（Pinsker的なやつです）を使うと，まず，
$$
\frac{1}{2}\left(f^{\star}\left(\pi^t\right)-\bar{f}^t\left(\pi^t\right)\right)^2 \leq D_{\mathrm{KL}}\left(\mathbb{P}_{\boldsymbol{r}^t \mid f^{\star}, \pi^t, \mathcal{H}^{t-1}} \| \mathbb{P}_{\boldsymbol{r}^t \mid \pi^t, \mathcal{H}^{t-1}}\right)
$$
が成立します．また，
$$
\frac{1}{2} \mathbb{E}_{f^{\star} \sim \mu^t} \mathbb{E}_{\pi^t \sim p^t}\left[\left(f^{\star}\left(\pi^t\right)-\bar{f}^t\left(\pi^t\right)\right)^2\right]\leq\mathbb{E}_{f^{\star} \sim \mu^t} \mathbb{E}_{\pi^t \sim p^t}\left[D_{\mathrm{KL}}\left(\mathbb{P}_{\boldsymbol{r}^t \mid f^{\star}, \pi^t, \mathcal{H}^{t-1}} \| \mathbb{P}_{\boldsymbol{r}^t \mid \pi^t, \mathcal{H}^{t-1}}\right)\right]
$$
も成立します．

ここで，
$\mathbb{E}_{x \sim \mathbb{P}_X}\left[D_{\mathrm{KL}}\left(\mathbb{P}_{Y \mid X=x} \| \mathbb{P}_Y\right)\right]=\mathbb{E}_{y \sim \mathbb{P}_Y}\left[D_{\mathrm{KL}}\left(\mathbb{P}_{X \mid Y=y} \| \mathbb{P}_X\right)\right]$であることを使うと（これは定義に当てはめたらすぐわかります），
右辺における$\boldsymbol{f}^\star$と$r^t$をひっくり返すことができて，右辺は
$$
\mathbb{E}_{t-1}\left[D_{\mathrm{KL}}\left(\mathbb{P}_{\boldsymbol{f}^{\star} \mid \pi^t, r^t, \mathcal{H}^{t-1}} \| \mathbb{P}_{\boldsymbol{f}^{\star} \mid \mathcal{H}^{t-1}}\right)\right]=\mathbb{E}_{t-1}\left[D_{\mathrm{KL}}\left(\mathbb{P}_{\boldsymbol{f}^{\star} \mid \mathcal{H}^t} \| \mathbb{P}_{\boldsymbol{f}^{\star} \mid \mathcal{H}^{t-1}}\right)\right]
$$
と等価です．更に右辺において$\mathcal{H}^{t-1}$の期待値を取ると，

$$
\mathbb{E}\left[\mathbb{E}_{t-1}\left[D_{\mathrm{KL}}\left(\mathbb{P}_{\boldsymbol{f}^{\star} \mid \mathcal{H}^t} \| \mathbb{P}_{\boldsymbol{f}^{\star} \mid \mathcal{H}^{t-1}}\right)\right]\right]=\mathbb{E}_{\mathcal{H}^{t-1}} \mathbb{E}_{\mathcal{H}^t \mid \mathcal{H}^{t-1}}\left[D_{\mathrm{KL}}\left(\mathbb{P}_{\boldsymbol{f}^{\star} \mid \mathcal{H}^t} \| \mathbb{P}_{\boldsymbol{f}^{\star} \mid \mathcal{H}^{t-1}}\right)\right]
$$

が成り立ちます．ここで，
$$
\mathbb{E}_{(x, y) \sim \mathbb{P}_{X, Y}}\left[D_{\mathrm{KL}}\left(\mathbb{P}_{Z \mid X=x, Y=y} \| \mathbb{P}_{Z \mid X=x}\right)\right]=\operatorname{Ent}(Z \mid X)-\operatorname{Ent}(Z \mid X, Y)
$$
であることを使えば，

$$
\mathbb{E}_{\mathcal{H}^{t-1}} \mathbb{E}_{\mathcal{H}^t \mid \mathcal{H}^{t-1}}\left[D_{\mathrm{KL}}\left(\mathbb{P}_{\boldsymbol{f}^{\star} \mid \mathcal{H}^t} \| \mathbb{P}_{\boldsymbol{f}^{\star} \mid \mathcal{H}^{t-1}}\right)\right]=\operatorname{Ent}\left(\boldsymbol{f}^{\star} \mid \mathcal{H}^{t-1}\right)-\operatorname{Ent}\left(\boldsymbol{f}^{\star} \mid \mathcal{H}^t\right)
$$
が得られます．
これはちゃんとポテンシャル関数として機能します．実際，
$$
\begin{aligned}
\frac{1}{2} \mathbb{E}\left[\sum_{t=1}^T \mathbb{E}_{f^{\star} \sim \mu^t} \mathbb{E}_{\pi^t \sim p^t}\left[\left(f^{\star}\left(\pi^t\right)-\bar{f}^t\left(\pi^t\right)\right)^2\right]\right] & =\sum_{t=1}^T \operatorname{Ent}\left(\boldsymbol{f}^{\star} \mid \mathcal{H}^{t-1}\right)-\operatorname{Ent}\left(\boldsymbol{f}^{\star} \mid \mathcal{H}^t\right) \\
& =\operatorname{Ent}\left(\boldsymbol{f}^{\star} \mid \mathcal{H}^0\right)-\operatorname{Ent}\left(\boldsymbol{f}^{\star} \mid \mathcal{H}^T\right) \\
& \leq \operatorname{Ent}\left(\boldsymbol{f}^{\star} \mid \mathcal{H}^0\right) \\
& \leq \log |\mathcal{F}|,
\end{aligned}
$$

より，リグレットがバウンドできます．

---

ここで見たベイズリグレットは，最初にやった頻度主義的なリグレットと関連付けることができます．
実際，Minimax定理を使えば，適切な条件のもと，次が成立します．

$$
\min _{\mathrm{Alg}} \max _{M^{\star}} \mathbb{E}^{M^{\star}, \mathrm{Alg}}[\mathbf{R e g}]=\max _{\mu \in \Delta(\mathcal{M})} \min _{\mathrm{Alg}} \mathbb{E}_{M^{\star} \sim \mu} \mathbb{E}^{M^{\star}, \text { Alg }}[\mathbf{R e g}]
$$

つまり，最悪ケースの事前分布を選択すると，それは頻度主義における最悪ケースのアルゴリズムの性能と一致します．


## 文脈付きバンディット

多腕バンディット問題では，有限個の独立したアームを考えていました．
しかし，現実的な状況ではアームに何かしらの情報がついています．
この章では文脈付きバンディットについて見てみましょう．
文脈付きバンディットでは次を繰り返します．

1. Observe context $x^t \in \mathcal{X}$.
2. Select decision $\pi^t \in \Pi=\{1, \ldots, A\}$. 
3. Observe reward $r^t \in \mathbb{R}$.

今回は，$x^t$が敵対的に出てくる状況について考えましょう．しかし，報酬は固定された分布から出ることを想定します．つまり，報酬は次のモデル$M^\star$からサンプルされるとします：
* $r^t \sim M^{\star}\left(\cdot \mid x^t, \pi^t\right)$

また，$\pi^{\star}(x):=\arg \max _{\pi \in \Pi} f^{\star}(x, \pi)$とします．
文脈付きバンディットでは，次のリグレットを考えます．

$$
\boldsymbol{R e g}:=\sum_{t=1}^T f^{\star}\left(x^t, \pi^{\star}\left(x^t\right)\right)-\sum_{t=1}^T \mathbb{E}_{\pi^t \sim p^t}\left[f^{\star}\left(x^t, \pi^t\right)\right]
$$

文脈の集合$\mathcal{X}$が有限な場合，最も簡単なやり方はそれぞれの$x$についてバンディットアルゴリズムを走らせるアプローチです．このアプローチでは，リグレットが$\widetilde{O}(\sqrt{A T \cdot|\mathcal{X}|})$になることを示すことができます．
これは$|\mathcal{X}|$に対して多項式的に依存してしまっているので嬉しくありません．そこで，期待報酬をモデリングする関数クラス$\mathcal{F}$を考え，$\log |\mathcal{F}|$にスケールするようなバウンドを出すことを目標にしましょう（統計的学習問題でも似たようなことをやりましたね）．
次を仮定します：

* $f^\star\in \mathcal{F}$を満たすクラス$\mathcal{F} \subset\{f: \mathcal{X} \times \Pi \rightarrow \mathbb{R}\}$にアクセス可能である．


**Optimismによるアプローチ**

多腕バンディット問題では，Optimismを使ったアプローチによって最適なオーダーのリグレットを導出することができました．
文脈付きバンディットでも，Optimismによって最適オーダーのリグレットは達成できるのでしょうか？これは問題に依存します．
一旦，Optimismによるアプローチのテンプレートを考えてみましょう．

次の信頼集合

$$\mathcal{F}^t \subseteq \mathcal{F}$$

を考えましょう．ここで，$f^\star \in \mathcal{F}^t$とします．
そして，この信頼集合に対して，信頼関数
$\underline{f}^t, \bar{f}^t: \mathcal{X} \times \Pi \rightarrow \mathbb{R}$
を次のように定めます．

$$
\underline{f}^t(x, \pi)=\min _{f \in \mathcal{F}^t} f(x, \pi), \quad \bar{f}^t(x, \pi)=\max _{f \in \mathcal{F}^t} f(x, \pi)
$$

realizabilityの性質より，

$$
\underline{f}^t(x, \pi) \leq f^{\star}(x, \pi) \leq \bar{f}^t(x, \pi)
$$

が任意の$x, \pi$について成立します．この信頼集合に対して，$\pi^t=\underset{\pi \in \Pi}{\arg \max } \bar{f}^t\left(x^t, \pi\right)$によって意思決定すると，
リグレットは
$$
\boldsymbol{R e g} \leq \sum_{t=1}^T \bar{f}^t\left(x^t, \pi^t\right)-\underline{f}^t\left(x^t, \pi^t\right)
$$
でバウンドされることがわかります．

さて，どのような信頼集合を作ればよいでしょうか？二乗誤差による信頼集合をまず考えてみましょう．

**二乗誤差による信頼集合**

$f(x, a) \in[0,1]$ および $r^t \in[0,1]$を考えます．そして，
$$
\widehat{f}^t=\underset{f \in \mathcal{F}}{\arg \min } \sum_{i=1}^{t-1}\left(f\left(x^i, \pi^i\right)-r^i\right)^2
$$
なる経験リスク最小化した予測機を考えます．$\beta:=8 \log (|\mathcal{F}| / \delta)$ および$\mathcal{F}^1=\mathcal{F}$として，

$$
\mathcal{F}^t=\left\{f \in \mathcal{F}: \sum_{i=1}^{t-1}\left(f\left(x^i, \pi^i\right)-r^i\right)^2 \leq \sum_{i=1}^{t-1}\left(\widehat{f^t}\left(x^i, \pi^i\right)-r^i\right)^2+\beta\right\}
$$

なる信頼集合を考えます．
つまり，信頼集合として，二乗誤差が$\hat{f}^t$によるものと十分近いものを考えます．
このような信頼集合について，次が成立します：

---

$\pi^1, \ldots, \pi^T$を適当な意思決定アルゴリズムとします．
このとき，確率$1-\delta$以上で，$f^{\star} \in \mathcal{F}^t$ for all $t \in[T]$が成立します．さらに，確率$1-\delta$以上で，任意の$\tau \leq T$と任意の$f \in \mathcal{F}^\tau$について，

$$
\sum_{t=1}^{\tau-1} \mathbb{E}_{\pi^t \sim p^t}\left[\left(f\left(x^t, \pi^t\right)-f^{\star}\left(x^t, \pi^t\right)\right)^2\right] \leq 4 \beta
$$

が成立します．

**証明**：（これはEluder dimensionのやつとほぼ同じっぽい）

$U^t(f)=\left(f\left(x^t, \pi^t\right)-r^t\right)^2-\left(f^{\star}\left(x^t, \pi^t\right)-r^t\right)^2$
とします．このとき，

$$
\mathbb{E}_{t-1} U^t(f)=\mathbb{E}_{t-1}\left(f\left(x^t, \pi^t\right)-f^{\star}\left(x^t, \pi^t\right)\right)^2
$$

が明らかに成立します．ここで，$\mathbb{E}_{t-1}[\cdot]:=\mathbb{E}\left[\cdot \mid \mathcal{H}^{t-1}, x^t\right]$
としました．

すると，$Z^t(f)=\mathbb{E}_{t-1} U^t(f)-U^t(f)$はマルチンゲール差分系列です．$\eta=1/8$とすると，Freedmanの不等式から，

$$
\sum_{t=1}^\tau Z^t(f) \leq \frac{1}{8} \sum_{t=1}^\tau \mathbb{E}_{t-1}\left[Z^t(f)^2\right]+8 \log \left(\delta^{-1}\right)
$$

が成立します．

TODO:証明を終わらせる．

---

ところで，上で示した補題だけではリグレットをバウンドするには不十分です．
実際，上の補題は$\left(x^1, \pi^1\right), \ldots,\left(x^{\tau-1}, \pi^{\tau-1}\right)$についての話ですが，リグレットのバウンドでは
$\left(x^\tau, \pi^\tau\right)$
における信頼区間の幅が要求されます．


### 線形バンディット

TODO: 書く


### Optimismが失敗するケース

線形バンディットではリグレットを抑えることができました．
ここでは，Optimismを使っても，$|\mathcal{F}|$もしくは$|\mathcal{X}|$にリグレットが依存してしまうケースを紹介します．

---

* $A=2$および$N\in \mathbb{N}$とします．
* $\Pi=\{\pi_g, \pi_b\}$の２つとします．
* $\mathcal{X}=\{x_1, \dots, x_N\}$を文脈とします
* $\mathcal{F}=\{f^\star, f_1, \dots, f_N\}$をモデルのクラスとします．
* $0 < \varepsilon < 1$とし，$f^\star(x, \pi_g)=1-\varepsilon$および$f^\star(x, \pi_b)=0$が任意の$x \in \mathcal{X}$で成立するとします．
    * つまり，$\pi_g$が常に最適な行動で，$1-\varepsilon$の報酬を返します．
    * $\pi_b$を選択すると，$1-\varepsilon$のリグレットが発生します．
* クラスの中の予測器：
    * それぞれの$i\in [N]$について，$f_i\left(x_j, \pi_g\right)=1-\varepsilon$
        * つまり，全ての予測器の$\pi_g$への予測は正しいとします
    * $j \neq i$について$f_i\left(x_j, \pi_b\right)=0$
        * つまり，文脈とは異なるインデックスの予測器では$\pi_b$の予測は正しいとします．
    * $f_i\left(x_i, \pi_g\right)=0$および$f_i\left(x_i, \pi_b\right)=1$とします．
        * つまり，文脈と同じインデックスの予測器では$\pi_b$の予測はOver estimateです．
* 決定的な報酬関数のバンディット問題を考えます：$r^t=f^{\star}\left(x^t, \pi^t\right)$

このとき，上でやったOptimismのアルゴリズムを使うと，新しい文脈に出会うと毎回$\pi_b$が選択され，$\Omega(N)$のリグレットが発生します．
これを証明しましょう．

$S^t$を，$t$ステップより以前に出会った異なる文脈の集合とします．
このとき，

$$
\widehat{f}^t=\underset{f \in \mathcal{F}}{\arg \min } \sum_{i=1}^{t-1}\left(f\left(x^i, \pi^i\right)-r^i\right)^2
$$

の解になり得るのは，$f^\star$と，$x_i\notin S^t$であるような$f_i$だけです．
すると，任意の$\beta\geq 0$について，信頼区間には$x_i\notin S^t$であるような$f_i$が含まれます．
よって，
$$
\bar{f}^t\left(x^t, \pi_b\right)=f_i\left(x_i, \pi_b\right)=1>\bar{f}^t\left(x^t, \pi_g\right)=f^{\star}\left(x^t, \pi_g\right)=1-\varepsilon .
$$
であるから，$x^t=x_i\notin S^t$であるような文脈に出会ったとき，かならず$\pi_b$が選択されます．

よって，文脈が番号順に提示される場合，リグレットは$\Omega(N(1-\varepsilon))$に従います．
故に，$|\mathcal{X}|=|\mathcal{F}|-1$なので，リグレットは$|\mathcal{F}|$に従ってしまいます．
また，この問題は文脈が一様分布に従って選択される場合でも発生します．（教科書参照）

---

教師あり学習では$|\mathcal{F}|$ではなく，$\log|\mathcal{F}|$にリグレットが従っていました．
上の例では，何が問題だったのでしょうか？
今回見た$\mathcal{F}$の構造は，「探索のしすぎ」をもたらしています．

線形バンディットと比べてみるとわかりやすいかもしれません．
線形バンディットでは，$\mathbb{R}^d$内の線形なクラスが，$d$個の異なる探索方向しか提示しません．
一方で，上の例では，新しい文脈に出会うたび，アルゴリズムは$\pi_b$が高い報酬を出すのではないか？という期待を持ち，探索を実行してしまいます．
これにより，信頼区間の幅が十分な速度で縮みません．


### 任意の$\mathcal{F}$でよく動く探索アルゴリズム

上で見たように，$\mathcal{F}$の構造次第で，Optimismは失敗する場合があります．
ここでは，任意の$\mathcal{F}$に対してlow regretになるアルゴリズムを見ていきます．
特に，$\mathcal{F}$を使った**オンライン学習アルゴリズム**を利用したアルゴリズムについて見ていきます．

**$\varepsilon$-greedy アルゴリズム**

次のオンライン回帰オラクルが存在することを仮定します．

---

**Online Regression Oracle**

$\mathbb{E}\left[r^i \mid x^i, \pi^i\right]=f^{\star}\left(x^i, \pi^i\right)$ かつ $\pi^i \sim p^i$であるような
$$
\left(x^1, \pi^1, r^1\right), \ldots,\left(x^{t-1}, \pi^{t-1}, r^{t-1}\right)
$$
サンプルが与えられるとします．ここで，$p^i=p^i\left(\cdot \mid x^i, \mathcal{H}^{i-1}\right)$は意思決定が従う分布です．

任意の$t\in [T]$について，オンライン回帰オラクルは，確率$1-\delta$以上で次を満たす関数$\widehat{f}^t: \mathcal{X} \times \Pi \rightarrow \mathbb{R}$を返すとします．
$$
\sum_{t=1}^T \mathbb{E}_{\pi^t \sim p^t}\left(\widehat{f^t}\left(x^t, \pi^t\right)-f^{\star}\left(x^t, \pi^t\right)\right)^2 \leq \operatorname{Est}_{\mathrm{Sq}}(\mathcal{F}, T, \delta)
$$

---

この仮定は比較的マイルドです．
例えば，上で学んだExponential Weights Algorithmでは，$\operatorname{Est}_{\mathrm{Sq}}(\mathcal{F}, T, \delta)=\log (|\mathcal{F}| / \delta)$です．

より一般的には，二乗誤差に対してリグレットバウンドを与える任意のアルゴリズムは，オンライン回帰オラクルになり得ます．
また，今回は文脈が敵対的に出ても良い，としているので，オンライン回帰オラクルを使おうとしていますが，もし文脈がi.i.d.にしか出てこないならば，統計的学習のオラクルを使えばよいです．

さて，$\varepsilon$-greedy アルゴリズムは，次を繰り返します．

1. $\left(x^1, \pi^1, r^1\right), \ldots,\left(x^{t-1}, \pi^{t-1}, r^{t-1}\right)$に対して，$\hat{f}^t$を計算します
2. $x^t$を得ます
3. 確率$\varepsilon$で$\pi^t \sim \operatorname{unif}([A])$を選び，確率$1-\varepsilon$で$\widehat{\pi}^t=\underset{\pi \in[A]}{\arg \max } \widehat{f}^t\left(x^t, \pi\right)$を選択します
4. $r^t$を得ます

この$\varepsilon$-greedyアルゴリズムに対して，次が成立します．

---

$f^{\star} \in \mathcal{F}$ and $f^{\star}(x, a) \in[0,1]$を仮定します．
このとき，$\varepsilon$-greedyアルゴリズムは，確率$1-\delta$以上で

$$
\boldsymbol{R e g} \lesssim A^{1 / 3} T^{2 / 3} \cdot \boldsymbol{\operatorname { E s t }}_{\mathrm{Sq}}(\mathcal{F}, T, \delta)^{1 / 3}
$$

のリグレットを任意の$x^1, \ldots, x^T$に対して達成します．特に$\mathcal{F}$が有限で，オンライン回帰オラクルとしてExponential Weights Algorithmを使った場合，
$$
\mathbf{R e g} \lesssim A^{1 / 3} T^{2 / 3} \cdot \log ^{1 / 3}(|\mathcal{F}| / \delta)
$$
のリグレットを達成します．

**証明**

まず，次のようにリグレットが分解できます：

$$
\begin{aligned}
\text { Reg } & =\sum_{t=1}^T \mathbb{E}_{\pi^t \sim p^t}\left[f^{\star}\left(\pi^{\star}\right)-f^{\star}\left(\pi^t\right)\right] \\
& =(1-\varepsilon) \sum_{t=1}^T f^{\star}\left(\pi^{\star}\right)-f^{\star}\left(\widehat{\pi}^t\right)+\varepsilon \sum_{t=1}^T \mathbb{E}_{\pi^t \sim \operatorname{unif}([A])}\left[f^{\star}\left(\pi^{\star}\right)-f^{\star}\left(\widehat{\pi}^t\right)\right] \\
& \leq \sum_{t=1}^T f^{\star}\left(\pi^{\star}\right)-f^{\star}\left(\widehat{\pi}^t\right)+\varepsilon T
\end{aligned}
$$

$t$を固定し，$\pi^{\star}\left(x^t\right)=\pi^{\star}$と表記すると，

$$
\begin{aligned}
& f^{\star}\left(x^t, \pi^{\star}\right)-f^{\star}\left(x^t, \widehat{\pi}^t\right) \\
& =\left[f^{\star}\left(x^t, \pi^{\star}\right)-\widehat{f}^t\left(x^t, \pi^{\star}\right)\right]+\left[\widehat{f}^t\left(x^t, \pi^{\star}\right)-\widehat{f}^t\left(x^t, \widehat{\pi}^t\right)\right]+\left[\widehat{f}^t\left(x^t, \widehat{\pi}^t\right)-f^{\star}\left(x^t, \widehat{\pi}^t\right)\right] \\
& \leq \sum_{\pi \in\left\{\widehat{\pi}^t, \pi^{\star}\right\}}\left|f^{\star}\left(x^t, \pi\right)-\widehat{f}^t\left(x^t, \pi\right)\right| \\
& =\sum_{\pi \in\left\{\widehat{\pi}^t, \pi^{\star}\right\}} \frac{1}{\sqrt{p^t(\pi)}} \sqrt{p^t(\pi)}\left|f^{\star}\left(x^t, \pi\right)-\widehat{f}^t\left(x^t, \pi\right)\right| .
\end{aligned}
$$

が成立します（不等式は貪欲方策のせいです）．
最後の項は，コーシー・シュワルツの不等式を使うと，

$$
\begin{aligned}
& \left\{\sum_{\pi \in\left\{\widehat{\pi}^t, \pi^{\star}\right\}} \frac{1}{p^t(\pi)}\right\}^{1 / 2}\left\{\sum_{\pi \in\left\{\widehat{\pi}^t, \pi^{\star}\right\}} p^t(\pi)\left(f^{\star}\left(x^t, \pi\right)-\widehat{f}^t\left(x^t, \pi\right)\right)^2\right\}^{1 / 2} \\
& \leq \sqrt{\frac{2 A}{\varepsilon}}\left\{\mathbb{E}_{\pi^t \sim p^t}\left(f^{\star}\left(x^t, \pi^t\right)-\widehat{f}^t\left(x^t, \pi^t\right)\right)^2\right\}^{1 / 2} .
\end{aligned}
$$

によってバウンドできます．$T$について和を取れば，

$$
\begin{aligned}
\sum_{t=1}^T f^{\star}\left(x^t, \pi^{\star}\left(x^t\right)\right)-f^{\star}\left(x^t, \widehat{\pi}^t\right) & \leq \sqrt{\frac{2 A}{\varepsilon}} \sum_{t=1}^T\left\{\mathbb{E}_{\pi^t \sim p^t}\left(f^{\star}\left(x^t, \pi^t\right)-\widehat{f}^t\left(x^t, \pi^t\right)\right)^2\right\}^{1 / 2} \\
& \leq \sqrt{\frac{2 A T}{\varepsilon}}\left\{\sum_{t=1}^T \mathbb{E}_{\pi^t \sim p^t}\left(f^{\star}\left(x^t, \pi^t\right)-\widehat{f}^t\left(x^t, \pi^t\right)\right)^2\right\}^{1 / 2}
\end{aligned}
$$

が成立します．
ここで，オンライン回帰オラクルは
$$
\sum_{t=1}^T \mathbb{E}_{\pi^t \sim p^t}\left(f^{\star}\left(x^t, \pi^t\right)-\widehat{f}^t\left(x^t, \pi^t\right)\right)^2 \leq \operatorname{Est}_{\mathrm{Sq}}(\mathcal{F}, T, \delta)
$$
を確率$1-\delta$以上で満たすので，

$$
\mathbf{R e g} \lesssim \sqrt{\frac{A T \mathbf{E s t}_{\mathrm{Sq}}(\mathcal{F}, T, \delta)}{\varepsilon}}+\varepsilon T
$$

が成立します．あとは$\varepsilon$を調整して終わりです．

---


**Inverse Gap Weighting アルゴリズム**

$\varepsilon$-greedyアルゴリズムではリグレットが$\sqrt{T}^{2/3}$になってしまいました．
今回は
$$
\boldsymbol{R e g} \lesssim \sqrt{A T \log |\mathcal{F}|}
$$
のアルゴリズムを見ていきます．

$\varepsilon$-Greedyアルゴリズムの解析のキモは，即時リグレット$\mathbb{E}_{\pi^t \sim p^t}\left[f^{\star}\left(x^t, \pi^{\star}\left(x^t\right)\right)-f^{\star}\left(x^t, \pi^t\right)\right]$を回帰誤差$\mathbb{E}_{\pi^t \sim p^t}\left[\left(f^{\star}\left(x^t, \pi^t\right)-\widehat{f}^t\left(x^t, \pi^t\right)\right)^2\right]$に関係させるステップにあります．

しかし，$\varepsilon$-Greedyアルゴリズムでは全ての行動に$\varepsilon/A$の確率を割り当てるので，余計なリグレットが発生してしまいました．
それを回避するために，次のInverse Gap Weightingを使います．

---

**Inverse Gap Weighting**

ベクトル$\widehat{f}=(\widehat{f}(1), \ldots, \widehat{f}(A)) \in \mathbb{R}^A$に対して，次を満たす分布$p=\operatorname{IGW}_\gamma(\widehat{f}(1), \ldots, \widehat{f}(A))$のことをInverse gap weighting分布と呼びます．

$$
p(\pi)=\frac{1}{\lambda+2 \gamma(\widehat{f}(\widehat{\pi})-\widehat{f}(\pi))}
$$

ここで$\gamma\geq 0$はパラメータで，$\widehat{\pi}=\arg \max _\pi \widehat{f}(\pi)$は貪欲方策，そして$\lambda \in [1, A]$は$\sum_\pi p(\pi)=1$になるように選択されたパラメータです（この$\lambda$は常に存在することが保証されます）．

**直感**

$\gamma$は探索と活用のパラメータを調整しているとみなせます．例えば$\gamma \to 0$ならば一様分布になり，$\gamma\to \infty$ならば貪欲方策が選択されます．

---

次の補題は，Inverse gap weightingによって即時リグレットと推定誤差が関連付けられることを示しています．

---

行動空間$\Pi=\{1, \ldots, A\}$を考えます．任意の$\hat{f}\in \mathbb{R}^A$と$\gamma > 0$に対して，$p=\operatorname{IGW}_\gamma(\widehat{f}(1), \ldots, \widehat{f}(A))$とします．
このとき，任意の$f^\star \in \mathbb{R}^A$について，

$$
\mathbb{E}_{\pi \sim p}\left[f^{\star}\left(\pi^{\star}\right)-f^{\star}(\pi)\right] \leq \frac{A}{\gamma}+\gamma \cdot \mathbb{E}_{\pi \sim p}\left[\left(\widehat{f}(\pi)-f^{\star}(\pi)\right)^2\right]
$$

が成立します．

**証明**

次の分解を考えます．

$$
\mathbb{E}_{\pi \sim p}\left[f^{\star}\left(\pi^{\star}\right)-f^{\star}(\pi)\right]=\underbrace{\mathbb{E}_{\pi \sim p}[\widehat{f}(\widehat{\pi})-\widehat{f}(\pi)]}_{\text {(I) exploration bias }}+\underbrace{\mathbb{E}_{\pi \sim p}\left[\widehat{f}(\pi)-f^{\star}(\pi)\right]}_{\text {(II) est. error on policy }}+\underbrace{f^{\star}\left(\pi^{\star}\right)-\widehat{f}(\widehat{\pi})}_{\text {(III) est. error at opt }}
$$

それぞれ見ていきましょう．
まず，最初の項は「もし$\hat{f}$が真の報酬関数ならば，Explorationによってどれくらい損するか？」を表しています．
この項は

$$
\begin{aligned}
(I) 
&= \sum_\pi \frac{\widehat{f}(\widehat{\pi})-\widehat{f}(\pi)}{\lambda+2 \gamma(\widehat{f}(\widehat{\pi})-\widehat{f}(\pi))} \\
&= \frac{\widehat{f}(\widehat{\pi})-\widehat{f}(\widehat{\pi})}{\lambda+2 \gamma(\widehat{f}(\widehat{\pi})-\widehat{f}(\widehat{\pi}))} 
+ \sum_{\pi\neq \widehat{\pi}} \frac{\widehat{f}(\widehat{\pi})-\widehat{f}(\pi)}{\lambda+2 \gamma(\widehat{f}(\widehat{\pi})-\widehat{f}(\pi))} \\
&= 0
+ \sum_{\pi\neq \widehat{\pi}} \frac{1}{\lambda / \left(\widehat{f}(\widehat{\pi})-\widehat{f}(\pi)\right)+2 \gamma} \\
&\leq \frac{A-1}{2 \gamma}\\
\end{aligned}
$$

でバウンドされます．

AM-GM不等式より，$\frac{x+y}{2} \geq \sqrt{x y}$を使えば，二項目は

$$
\begin{aligned}
(II) 
&\leq \sqrt{\mathbb{E}_{\pi \sim p}\left[\left(\widehat{f}(\pi)-f^{\star}(\pi)\right)^2\right]} \\
&\leq \frac{1}{2 \gamma}+\frac{\gamma}{2} \mathbb{E}_{\pi \sim p}\left(\widehat{f}(\pi)-f^{\star}(\pi)\right)^2\\
\end{aligned}
$$

でバウンドされます．

三項目は，
$$
\begin{aligned}
(III) = 
f^{\star}\left(\pi^{\star}\right)-\widehat{f}\left(\pi^{\star}\right)-\left(\widehat{f}(\widehat{\pi})-\widehat{f}\left(\pi^{\star}\right)\right) & \leq \frac{\gamma}{2} p\left(\pi^{\star}\right)\left(f^{\star}\left(\pi^{\star}\right)-\widehat{f}\left(\pi^{\star}\right)\right)^2+\frac{1}{2 \gamma p\left(\pi^{\star}\right)}-\left(\widehat{f}(\widehat{\pi})-\widehat{f}\left(\pi^{\star}\right)\right) \\
& \leq \frac{\gamma}{2} \mathbb{E}_{\pi \sim p}\left(f^{\star}(\pi)-\widehat{f}(\pi)\right)^2+\left[\frac{1}{2 \gamma p\left(\pi^{\star}\right)}-\left(\widehat{f}(\widehat{\pi})-\widehat{f}\left(\pi^{\star}\right)\right)\right]
\end{aligned}
$$

でバウンドされます．最初の不等式では，二項目のAM-GM不等式と似たようなことをやっているだけです．

ここで，三項目の$[]$の中身は
$$
\frac{\lambda+2 \gamma\left(\widehat{f}(\widehat{\pi})-\widehat{f}\left(\pi^{\star}\right)\right)}{2 \gamma}-\left(\widehat{f}(\widehat{\pi})-\widehat{f}\left(\pi^{\star}\right)\right)=\frac{\lambda}{2 \gamma} \leq \frac{A}{2 \gamma}
$$

によってバウンドされます．あとは全部合体させて終わりです．

---

この補題はかなり便利で，この後もめっちゃ使います．
さて，このInverse Gap Weightingの性質を使ったSquareCBアルゴリズムを見てみましょう．

**SquareCBアルゴリズム**

これは次を繰り返します．

1. $\left(x^1, \pi^1, r^1\right), \ldots,\left(x^{t-1}, \pi^{t-1}, r^{t-1}\right)$に対して，$\hat{f}^t$を計算します
2. $x^t$を得ます
3. $p^t=\operatorname{IGW}_\gamma\left(\widehat{f}^t\left(x^t, 1\right), \ldots, \widehat{f}^t\left(x^t, A\right)\right)$を計算します．
4. $\pi^t \sim p^t$で意思決定します．
5. $r^t$を得ます

SquareCBアルゴリズムのリグレットを導出しましょう．
まず，

$$
\begin{aligned}
\mathbf{R e g} & =\sum_{t=1}^T \mathbb{E}_{\pi^t \sim p^t}\left[f^{\star}\left(x^t, \pi^{\star}\right)-f^{\star}\left(x^t, \pi^t\right)\right] \\
& \leq\sum_{t=1}^T \mathbb{E}_{\pi^t \sim p^t}\left[f^{\star}\left(x^t, \pi^{\star}\right)-f^{\star}\left(x^t, \pi^t\right)-\gamma \cdot\left(f^{\star}\left(x^t, \pi^t\right)-\widehat{f}^t\left(x^t, \pi^t\right)\right)^2\right]+\gamma \cdot \mathbf{E s t}_{\mathrm{Sq}}(\mathcal{F}, T, \delta)
\end{aligned}
$$

が成立します．１項目については，上で学んだ補題を使えば，

$$
\mathbb{E}_{\pi^t \sim p^t}\left[f^{\star}\left(x^t, \pi^{\star}\right)-f^{\star}\left(x^t, \pi^t\right)-\gamma \cdot\left(f^{\star}\left(x^t, \pi^t\right)-\widehat{f}^t\left(x^t, \pi^t\right)\right)^2\right] \leq \frac{A}{\gamma}
$$

で抑えられます．よって，

$$
\boldsymbol{R e g} \leq \frac{T A}{\gamma}+\gamma \cdot \boldsymbol{E s t}_{\mathrm{Sq}}(\mathcal{F}, T, \delta)
$$

が成立します．あとは$\gamma$を調整すると，
$$
\boldsymbol{\operatorname { R e g }} \lesssim \sqrt{A T \operatorname{Est}_{\mathrm{Sq}}(\mathcal{F}, T, \delta)}
$$
が得られます．特に$\mathcal{F}$が有限で，オラクルとしてaveraged exponential weights アルゴリズムを使えば，

$$
\mathbf{R e g} \lesssim \sqrt{A T \log (|\mathcal{F}| / \delta})
$$

が成立します．

---

**オフライン版のSquareCBアルゴリズム**

TODO: 書く



## 構造化バンディット問題

